In [1]:
import pandas as pd
import numpy as np

# Replace with personal path to run
PATH = ''

df_games = pd.read_csv(PATH + 'games.csv', index_col=0)
df_players = pd.read_csv(PATH + 'players.csv', index_col=0)
df_plays = pd.read_csv(PATH + 'plays.csv')
df_week1 = pd.read_csv(PATH + 'week1.csv')

# Part 1

In [2]:
# Setup
df_part1 = df_week1[['time', 'x', 'y', 'event', 'nflId', 'frameId', 'gameId', 'playId']].sort_values(['gameId', 'playId', 'nflId', 'frameId'])

# Calc dis
df_part1['prevX'], df_part1['prevY'] = np.concatenate([[[0, 0]], df_part1[['x', 'y']].values[0:len(df_part1) - 1]]).T
df_part1['disCalc'] = np.where(df_part1['frameId'] > 1, pow(pow(df_part1['x'] - df_part1['prevX'], 2) + pow(df_part1['y'] - df_part1['prevY'], 2), .5), 0)

# Calc after snap dis
df_snapTimes = df_week1.loc[(df_week1['nflId'].isnull()) & (df_week1['event'] == 'ball_snap')][['frameId', 'gameId', 'playId']]
df_snapTimes.columns = ['snapFrameId', 'gameId', 'playId']
df_part1 = pd.merge(df_part1, df_snapTimes, how='left', on=['gameId', 'playId'])
df_part1['disCalcAs'] = np.where(df_part1['frameId'] >= df_part1['snapFrameId'], df_part1['disCalc'], 0)

# Calc s
MAX_SPEED = 11.5
df_part1['prevTime'] = np.concatenate([[['2018-01-01T01:07:14.599Z']], df_part1[['time']].values[0:len(df_part1) - 1]]).T[0]
df_part1['timeDif'] = (pd.to_datetime(df_part1['time']) - pd.to_datetime(df_part1['prevTime'])).dt.total_seconds()
df_part1['sCalc'] = np.where((df_part1['frameId'] > 1) & (df_part1['timeDif'] > 0), np.minimum(df_part1['disCalc'] / df_part1['timeDif'], MAX_SPEED), 0)

# Smooth speed calculations to address variability in instantaneous 
SMOOTH_CONSTANT = 2 # number of speeds before/after to average
df_part1['sSum'] = df_part1['sCalc']
df_part1['sCount'] = 1
for i in range(SMOOTH_CONSTANT):
    df_part1['sTemp'] = np.concatenate([[[0] for j in range(i + 1)], df_part1[['sCalc']].values[0:len(df_part1) - i - 1]]).T[0]
    df_part1['sSum'] = np.where(df_part1['frameId'] > i + 1, df_part1['sSum'] + df_part1['sTemp'], df_part1['sSum'])
    df_part1['sCount'] = np.where(df_part1['frameId'] > i + 1, df_part1['sCount'] + 1, df_part1['sCount'])

df_lastFrameId = df_week1[['gameId', 'playId', 'frameId']].groupby(['gameId', 'playId']).max().reset_index()
df_lastFrameId.columns = ['gameId', 'playId', 'lastFrameId']
df_part1 = pd.merge(df_part1, df_lastFrameId, how='left', on=['gameId', 'playId'])

for i in range(SMOOTH_CONSTANT):
    df_part1['sTemp'] = np.concatenate([df_part1[['sCalc']].values[i + 1:len(df_part1)], [[0] for j in range(i + 1)]]).T[0]
    df_part1['sSum'] = np.where(df_part1['frameId'] + i < df_part1['lastFrameId'], df_part1['sSum'] + df_part1['sTemp'], df_part1['sSum'])
    df_part1['sCount'] = np.where(df_part1['frameId'] + i < df_part1['lastFrameId'], df_part1['sCount'] + 1, df_part1['sCount'])

df_part1['sSmooth'] = df_part1['sSum'] / df_part1['sCount']

### Total distance

In [3]:
# Total distance results
df_totalDistance = df_part1[['nflId', 'disCalc']].groupby(['nflId']).sum().join(df_players[['displayName']]).sort_values('disCalc', ascending=False)
df_totalDistance.columns = ['Total Distance', 'Player Name']

print('Player who traveled the most distance in week 1:\n')
print(df_totalDistance['Player Name'][0] + ': ' + str(round(df_totalDistance['Total Distance'][0], 2)) + ' yards')

df_totalDistance

Player who traveled the most distance in week 1:

T.Y. Hilton: 1306.51 yards


,Total Distance,Player Name
nflId,,
2532865.0,1306.510265,T.Y. Hilton
2508061.0,1302.443109,Antonio Brown
2558052.0,1300.916768,Kenny Golladay
2532884.0,1178.050317,Marvin Jones
2556617.0,1141.431661,Doug Middleton
...,...,...
2559044.0,8.757425,Antony Auclair
2536044.0,7.705823,Jamize Olawale
2556414.0,6.120009,Rob Kelley


### Single play distance

In [4]:
# Single play distance results
df_singlePlayDistance = df_part1[['nflId', 'playId', 'disCalc']].groupby(['nflId', 'playId']).sum().join(df_players[['displayName']]).sort_values('disCalc', ascending=False)
df_singlePlayDistance.columns = ['Play Distance', 'Player Name']

print('Player who traveled the most distance in a single play:\n')
print(df_singlePlayDistance['Player Name'][0] + ': ' + str(round(df_singlePlayDistance['Play Distance'][0], 2)) + ' yards')

df_singlePlayDistance

Player who traveled the most distance in a single play:

Geronimo Allison: 105.16 yards


,,Play Distance,Player Name
nflId,playId,,
2556460.0,3564,105.158383,Geronimo Allison
2555355.0,4219,102.109112,Clayton Fejedelem
2552429.0,3564,102.011920,Ty Montgomery
2495448.0,3564,99.732174,Randall Cobb
2532903.0,3578,97.984403,Rishard Matthews
...,...,...,...
2543488.0,5449,0.084142,Jarvis Landry
2560894.0,1982,0.078284,Ian Thomas
2555536.0,1982,0.078284,Kavon Frazier


### Fastest speeds
Due to the variability of instantaneous speed calculations, I decided to 'smooth' speed calculations by averaging each speed value by its predecessors and successors. This resulted in slightly lower max speeds, but a much more accurate ranking.

In [5]:
# Fastest speed results
df_fastestSpeeds = df_part1[['nflId', 'playId', 'sSmooth']].groupby(['nflId', 'playId']).max().join(df_players[['displayName']]).sort_values('sSmooth', ascending=False)
df_fastestSpeeds.columns = ['Fastest Play Speed', 'Player Name']

print('Fastest week 1 speeds:\n')
for i in range(5):
    print(df_fastestSpeeds['Player Name'][i] + ': ' + str(round(df_fastestSpeeds['Fastest Play Speed'][i], 2)) + ' yards/second')

df_fastestSpeeds

Fastest week 1 speeds:

Derrick Kindred: 10.89 yards/second
Jamie Collins: 10.85 yards/second
Marcus Williams: 10.63 yards/second
William Jackson: 10.57 yards/second
Christian Kirksey: 10.54 yards/second


,,Fastest Play Speed,Player Name
nflId,playId,,
2555540.0,5369,10.891769,Derrick Kindred
2539311.0,1912,10.851930,Jamie Collins
2558847.0,354,10.626066,Marcus Williams
2556367.0,4096,10.566647,William Jackson
2543720.0,4020,10.537147,Christian Kirksey
...,...,...,...
382.0,2696,0.037712,Joe Flacco
497095.0,3390,0.034928,Sam Bradford
2553796.0,3390,0.031623,Quinton Dunbar


<br>
<br>
<br>
<br>
<br>
<br>

# Part 2

### Calculate each player's max speed for the week
Same code as in part 1 except results are grouped by nflId instead of nflId and playId (returns max speed across all plays for a player)

In [6]:
# Setup
df_test = df_week1[['time', 'x', 'y', 'event', 'nflId', 'frameId', 'gameId', 'playId']].sort_values(['gameId', 'playId', 'nflId', 'frameId']).drop_duplicates(['gameId', 'playId', 'nflId', 'frameId', 'time'])

# Calc dis
df_test['prevX'], df_test['prevY'] = np.concatenate([[[0, 0]], df_test[['x', 'y']].values[0:len(df_test) - 1]]).T
df_test['disCalc'] = np.where(df_test['frameId'] > 1, pow(pow(df_test['x'] - df_test['prevX'], 2) + pow(df_test['y'] - df_test['prevY'], 2), .5), 0)

# Calc s
MAX_SPEED = 11.5
df_test['prevTime'] = np.concatenate([[['2018-01-01T01:07:14.599Z']], df_test[['time']].values[0:len(df_test) - 1]]).T[0]
df_test['timeDif'] = (pd.to_datetime(df_test['time']) - pd.to_datetime(df_test['prevTime'])).dt.total_seconds()
df_test['sCalc'] = np.where((df_test['frameId'] > 1) & (df_test['timeDif'] > 0), np.minimum(df_test['disCalc'] / df_test['timeDif'], MAX_SPEED), 0)

# Smooth s
SMOOTH_CONSTANT = 2 # number of speeds before/after to average
df_test['sSum'] = df_test['sCalc']
df_test['sCount'] = 1
for i in range(SMOOTH_CONSTANT):
    df_test['sTemp'] = np.concatenate([[[0] for j in range(i + 1)], df_test[['sCalc']].values[0:len(df_test) - i - 1]]).T[0]
    df_test['sSum'] = np.where(df_test['frameId'] > i + 1, df_test['sSum'] + df_test['sTemp'], df_test['sSum'])
    df_test['sCount'] = np.where(df_test['frameId'] > i + 1, df_test['sCount'] + 1, df_test['sCount'])

# Add last frameId
df_lastFrameId = df_week1[['gameId', 'playId', 'frameId']].groupby(['gameId', 'playId']).max().reset_index()
df_lastFrameId.columns = ['gameId', 'playId', 'lastFrameId']
df_test = pd.merge(df_test, df_lastFrameId, how='left', on=['gameId', 'playId'])

for i in range(SMOOTH_CONSTANT):
    df_test['sTemp'] = np.concatenate([df_test[['sCalc']].values[i + 1:len(df_test)], [[0] for j in range(i + 1)]]).T[0]
    df_test['sSum'] = np.where(df_test['frameId'] + i < df_test['lastFrameId'], df_test['sSum'] + df_test['sTemp'], df_test['sSum'])
    df_test['sCount'] = np.where(df_test['frameId'] + i < df_test['lastFrameId'], df_test['sCount'] + 1, df_test['sCount'])

df_test['sSmooth'] = df_test['sSum'] / df_test['sCount']

df_playerSpeeds = df_test[['nflId', 'sSmooth']].groupby(['nflId']).max()
df_playerSpeeds.columns = ['maxSpeed']

### Metric calculations

In [7]:
# setup
df_part2 = df_week1[['time', 'x', 'y', 'event', 'nflId', 'frameId', 'team', 'gameId', 'playId']].sort_values(['gameId', 'playId', 'nflId', 'frameId'])

# Add distance to ball for each player
df_ballLocations = df_part2[['gameId', 'playId', 'frameId', 'x', 'y']].loc[df_week1['nflId'].isnull()]
df_ballLocations.columns = ['gameId', 'playId', 'frameId', 'ballX', 'ballY']
df_part2 = pd.merge(df_part2, df_ballLocations, how='left', on=['gameId', 'playId', 'frameId'])

# Add if player is offensive or defensive
df_pos = df_plays[['gameId', 'playId', 'possessionTeam']]
df_pos = pd.merge(df_pos, df_games[['homeTeamAbbr']], how='left', on=['gameId'])
df_pos['pos'] = np.where(df_pos['possessionTeam'] == df_pos['homeTeamAbbr'], 'home', 'away')
df_pos = df_pos.drop(['possessionTeam', 'homeTeamAbbr'], axis=1)
df_part2 = pd.merge(df_part2, df_pos, how='left', on=['gameId', 'playId'])
df_part2['sob'] = np.where(df_part2['team'] == df_part2['pos'], 'off', 'def')
df_part2 = df_part2.drop(['team', 'pos'], axis=1)

# caught to tackle/out of bounds/touchdown/fumble
df_end = df_part2.loc[(df_part2['event'] == 'tackle') | (df_part2['event'] == 'out_of_bounds') | (df_part2['event'] == 'touchdown') | (df_part2['event'] == 'fumble')][['gameId', 'playId', 'frameId', 'nflId', 'time', 'x', 'y', 'ballX', 'ballY', 'sob']].sort_values('frameId').drop_duplicates(['gameId', 'playId', 'nflId'], keep = 'first')
df_end.columns = ['gameId', 'playId', 'endFrameId', 'nflId', 'timeEnd', 'xEnd', 'yEnd', 'ballXEnd', 'ballYEnd', 'sob']
df_start = df_part2.loc[df_part2['event'] == 'pass_outcome_caught'][['gameId', 'playId', 'frameId', 'nflId', 'time', 'x', 'y', 'ballX', 'ballY']]
df_start.columns = ['gameId', 'playId', 'startFrameId', 'nflId', 'timeStart', 'xStart', 'yStart', 'ballXStart', 'ballYStart']
df_hustle = pd.merge(df_start, df_end, how='left', on=['gameId', 'playId', 'nflId'])

# interception/fumble to tackle/out of bounds/touchdown
df_end = df_part2.loc[(df_part2['event'] == 'tackle') | (df_part2['event'] == 'out_of_bounds') | (df_part2['event'] == 'touchdown') | (df_part2['event'] == 'touchback')][['gameId', 'playId', 'frameId', 'nflId', 'time', 'x', 'y', 'ballX', 'ballY', 'sob']].sort_values('frameId').drop_duplicates(['gameId', 'playId', 'nflId'], keep = 'first')
df_end.columns = ['gameId', 'playId', 'endFrameId', 'nflId', 'timeEnd', 'xEnd', 'yEnd', 'ballXEnd', 'ballYEnd', 'sob']
df_start = df_part2.loc[(df_part2['event'] == 'pass_outcome_interception') | (df_part2['event'] == 'fumble_defense_recovered')][['gameId', 'playId', 'frameId', 'nflId', 'time', 'x', 'y', 'ballX', 'ballY']]
df_start.columns = ['gameId', 'playId', 'startFrameId', 'nflId', 'timeStart', 'xStart', 'yStart', 'ballXStart', 'ballYStart']
df_hustle = df_hustle.append(pd.merge(df_start, df_end, how='left', on=['gameId', 'playId', 'nflId']))
df_hustle['type'] = 'after catch'

# before catch (defense only)
df_end = df_part2.loc[df_part2['event'] == 'pass_arrived'][['gameId', 'playId', 'frameId', 'nflId', 'time', 'x', 'y', 'ballX', 'ballY', 'sob']]
df_end.columns = ['gameId', 'playId', 'endFrameId', 'nflId', 'timeEnd', 'xEnd', 'yEnd', 'ballXEnd', 'ballYEnd', 'sob']
df_start = df_part2.loc[(df_part2['event'] == 'pass_forward') | (df_part2['event'] == 'pass_shovel')][['gameId', 'playId', 'frameId', 'nflId', 'time', 'x', 'y']]
df_start.columns = ['gameId', 'playId', 'startFrameId', 'nflId', 'timeStart', 'xStart', 'yStart']
df_beforeCatch = pd.merge(df_end, df_start, how='left', on=['gameId', 'playId', 'nflId'])
df_beforeCatch['type'] = 'before catch'
df_hustle = df_hustle.append(df_beforeCatch)

# distances
df_hustle['disStart'] = pow(pow(df_hustle['xStart'] - df_hustle['ballXEnd'], 2) + pow(df_hustle['yStart'] - df_hustle['ballYEnd'], 2), .5)
df_hustle['disEnd'] = pow(pow(df_hustle['xEnd'] - df_hustle['ballXEnd'], 2) + pow(df_hustle['yEnd'] - df_hustle['ballYEnd'], 2), .5)
df_hustle['disDif'] = df_hustle['disStart'] - df_hustle['disEnd']
df_hustle['disToStart'] = pow(pow(df_hustle['xStart'] - df_hustle['ballXStart'], 2) + pow(df_hustle['yStart'] - df_hustle['ballYStart'], 2), .5)

# before catch distances
df_targetted = df_hustle.loc[df_hustle['sob'] == 'off'].sort_values(['gameId', 'playId', 'disEnd']).drop_duplicates(['gameId', 'playId'], keep = 'first')[['gameId', 'playId', 'nflId', 'xStart', 'yStart']]
df_targetted.columns = ['gameId', 'playId', 'targetId', 'xTarget', 'yTarget']
df_hustle = pd.merge(df_hustle, df_targetted, how='left', on=['gameId', 'playId'])
df_hustle['targetDisStart'] = pow(pow(df_hustle['xStart'] - df_hustle['xTarget'], 2) + pow(df_hustle['yStart'] - df_hustle['yTarget'], 2), .5)

# add hustle info
df_hustle = pd.merge(df_hustle, df_playerSpeeds, how='left', on=['nflId'])
df_hustle['timeDif'] = (pd.to_datetime(df_hustle['timeEnd']) - pd.to_datetime(df_hustle['timeStart'])).dt.total_seconds()
df_hustle['hustleOpp'] = np.where(((df_hustle['type'] == 'before catch') & (df_hustle['sob'] == 'def') & (df_hustle['targetDisStart'] > 5)) | ((df_hustle['type'] == 'after catch') & (df_hustle['disToStart'] > 5)), np.minimum(df_hustle['maxSpeed'] * df_hustle['timeDif'], df_hustle['disStart']), 0)
df_hustle['hustleYards'] = np.where((df_hustle['type'] == 'after catch') | ((df_hustle['type'] == 'before catch') & (df_hustle['sob'] == 'def')), np.minimum(df_hustle['disDif'], df_hustle['hustleOpp']), 0)
df_hustle['hustlePercent'] = np.where(df_hustle['hustleOpp'] > 0, df_hustle['hustleYards'] / df_hustle['hustleOpp'], 0)

c:\users\jackson\documents\cse 494 (machine learning)\code\old\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


### Metric results

#### Defense

In [8]:
SIDE_OF_BALL = 'def'

# total hustle opportunity yards
df_totalOpp = df_hustle.loc[(df_hustle['nflId'].notnull()) & (df_hustle['sob'] == SIDE_OF_BALL)][['nflId', 'hustleOpp']].groupby(['nflId']).sum()

# play count
df_totalPlays = df_hustle.loc[(df_hustle['nflId'].notnull()) & (df_hustle['sob'] == SIDE_OF_BALL) & (df_hustle['hustleOpp'] > 0)][['nflId', 'hustleYards']].groupby(['nflId']).count()
df_totalPlays.columns = ['playCount']

# total hustle yards
df_totalHustleYards = df_hustle.loc[(df_hustle['nflId'].notnull()) & (df_hustle['sob'] == SIDE_OF_BALL)][['nflId', 'hustleYards']].groupby(['nflId']).sum().join(df_totalPlays).join(df_totalOpp).join(df_players[['displayName', 'position']]).sort_values('hustleYards', ascending=False)

# hustle rating
df_totalHustleYards['hustlePercent'] = df_totalHustleYards['hustleYards'] / df_totalHustleYards['hustleOpp']
df_totalHustleYards['hustleRating'] = df_totalHustleYards['hustlePercent'] * pow(df_totalHustleYards['playCount'], .5)
df_totalHustleYards.columns = ['Hustle Yards', 'Play Count', 'Opportunity Yards', 'Player Name', 'Position', 'Hustle Percent', 'Hustle Rating']

# print results
pd.set_option('display.max_rows', 300)
df_totalHustleYards.loc[df_totalHustleYards['Play Count'] >= 1].sort_values('Hustle Rating', ascending=False)

,Hustle Yards,Play Count,Opportunity Yards,Player Name,Position,Hustle Percent,Hustle Rating
nflId,,,,,,,
2560789.0,286.350547,41.0,467.918271,Darius Leonard,LB,0.611967,3.918501
2556617.0,334.901185,78.0,792.764448,Doug Middleton,S,0.422447,3.730953
2560970.0,290.131287,82.0,713.969429,Jessie Bates,S,0.406364,3.679780
2539935.0,290.138693,52.0,602.423028,Kiko Alonso,OLB,0.481620,3.473008
2556277.0,211.233744,41.0,400.256713,Marqui Christian,SS,0.527746,3.379221
2555355.0,266.607940,72.0,687.768474,Clayton Fejedelem,FS,0.387642,3.289251
2552386.0,252.482103,43.0,512.235077,Clayton Geathers,FS,0.492903,3.232180
2558070.0,244.924533,42.0,495.467321,Raekwon McMillan,LB,0.494330,3.203627
2530491.0,207.674244,34.0,381.504085,Josh Bynes,ILB,0.544357,3.174117


#### Offense

In [9]:
SIDE_OF_BALL = 'off'

# total hustle opportunity yards
df_totalOpp = df_hustle.loc[(df_hustle['nflId'].notnull()) & (df_hustle['sob'] == SIDE_OF_BALL)][['nflId', 'hustleOpp']].groupby(['nflId']).sum()

# play count
df_totalPlays = df_hustle.loc[(df_hustle['nflId'].notnull()) & (df_hustle['sob'] == SIDE_OF_BALL) & (df_hustle['hustleOpp'] > 0)][['nflId', 'hustleYards']].groupby(['nflId']).count()
df_totalPlays.columns = ['playCount']

# total hustle yards
df_totalHustleYards = df_hustle.loc[(df_hustle['nflId'].notnull()) & (df_hustle['sob'] == SIDE_OF_BALL)][['nflId', 'hustleYards']].groupby(['nflId']).sum().join(df_totalPlays).join(df_totalOpp).join(df_players[['displayName', 'position']]).sort_values('hustleYards', ascending=False)

# hustle rating
df_totalHustleYards['hustlePercent'] = df_totalHustleYards['hustleYards'] / df_totalHustleYards['hustleOpp']
df_totalHustleYards['hustleRating'] = df_totalHustleYards['hustlePercent'] * pow(df_totalHustleYards['playCount'], .5)
df_totalHustleYards.columns = ['Hustle Yards', 'Play Count', 'Opportunity Yards', 'Player Name', 'Position', 'Hustle Percent', 'Hustle Rating']

# print results
pd.set_option('display.max_rows', 300)
df_totalHustleYards.loc[df_totalHustleYards['Play Count'] >= 1].sort_values('Hustle Rating', ascending=False)

,Hustle Yards,Play Count,Opportunity Yards,Player Name,Position,Hustle Percent,Hustle Rating
nflId,,,,,,,
2558207.0,67.288666,11.0,121.268630,Jonnu Smith,TE,0.554873,1.840305
2556466.0,44.177396,13.0,86.993000,Peyton Barber,RB,0.507827,1.830996
2532903.0,124.294120,13.0,249.308956,Rishard Matthews,WR,0.498555,1.797564
2557848.0,147.675168,16.0,331.345570,Corey Davis,WR,0.445683,1.782733
497236.0,138.720925,22.0,375.627396,Jimmy Graham,TE,0.369305,1.732192
2555364.0,84.661562,15.0,190.584985,Tyler Higbee,TE,0.444219,1.720455
2557978.0,172.729888,19.0,449.194120,James Conner,RB,0.384533,1.676140
2552559.0,99.319659,12.0,208.565867,C.J. Uzomah,TE,0.476203,1.649615
2560933.0,76.248506,18.0,201.519274,Jordan Wilkins,RB,0.378368,1.605281
